In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tarfile
import urllib
from scipy import ndimage
#from sklearn.linear_model import LogisticRegression
import cPickle as pickle
import tensorflow as tf
import math

In [2]:
image_size = 64  # Pixel width and height.
pixel_depth = 255.0  # Number of levels per pixel.
num_labels = 2
num_channels = 3 # grayscale

testFolders = ['/Users/meicg/Documents/MATLAB/label_64_1'];
originalFolder = ['/Users/meicg/Documents/MATLAB/label_1']
classifyFolders = ['/Users/meicg/Documents/MATLAB/auto_classify_0',
                   '/Users/meicg/Documents/MATLAB/auto_classify_1']

In [3]:
def loadTestImages(data_folders,min_num_images, max_num_images):
  dataset = np.ndarray(
    shape=(max_num_images, image_size, image_size, num_channels), dtype=np.float32)
  image_index = 0
  for folder in data_folders:
    print folder
    for image in os.listdir(folder):
      image_file = os.path.join(folder, image)
      try:
        image_data = (ndimage.imread(image_file).astype(float) -
                      pixel_depth / 2) / pixel_depth
        if image_data.shape != (image_size, image_size, num_channels):
          raise Exception('Unexpected image shape: %s' % str(image_data.shape))
        dataset[image_index, :, :, :] = image_data
        image_index += 1
      except IOError as e:
        print 'Could not read:', image_file, ':', e, '- it\'s ok, skipping.'
  num_images = image_index
  dataset = dataset[0:num_images, :, :, :]
  label_result = np.ndarray(shape=(num_images, num_labels ), dtype=np.float32)
  print 'Full dataset tensor:', dataset.shape
  print 'Mean:', np.mean(dataset)
  print 'Standard deviation:', np.std(dataset)
  return dataset,label_result 

testDataSet,label_result = loadTestImages(testFolders, 6000, 7000)

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

/Users/meicg/Documents/MATLAB/label_64_1


OSError: [Errno 2] No such file or directory: '/Users/meicg/Documents/MATLAB/label_64_1'

In [7]:
#batch_size = 128
calculate_image_per_times = 1
patch_size = 5
depth = 16
num_hidden = 512
#lamada = 0.02
graph = tf.Graph()

with graph.as_default():

  # Input data.
  #tf_train_dataset = tf.placeholder(
  #  tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  #tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  #tf_valid_dataset = tf.constant(valid_dataset)
  #tf_test_dataset = tf.constant(test_dataset)
  tf_classify_dataset = tf.placeholder(
    tf.float32, shape=(calculate_image_per_times, image_size, image_size, num_channels))
  
  # Variables.
  conv1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  conv1_biases = tf.Variable(tf.zeros([depth]))
  conv2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth*2], stddev=0.1))
  conv2_biases = tf.Variable(tf.constant(0.0, shape=[depth*2]))
  conv3_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth*2, depth*4], stddev=0.1))
  conv3_biases = tf.Variable(tf.constant(0.0, shape=[depth*4]))
  conv4_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth*4, depth*8], stddev=0.1))
  conv4_biases = tf.Variable(tf.constant(1.0, shape=[depth*8]))


  full_weights = tf.Variable(tf.truncated_normal(
      [image_size / 16 * image_size / 16 * depth*8, num_hidden], stddev=0.1))
  full_biases = tf.Variable(tf.constant(0.0, shape=[num_hidden]))
  softmax_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  softmax_biases = tf.Variable(tf.constant(0.0, shape=[num_labels]))

  # Dropout.
  keep_prob = tf.placeholder("float")
  
  # Model.
  def model(data):
    conv_1 = tf.nn.conv2d(data, conv1_weights, [1, 1, 1, 1], padding='SAME')
    relu_1 = tf.nn.relu(conv_1 + conv1_biases)
    max_pool_1 = tf.nn.max_pool(relu_1, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1],padding = 'SAME')
    
    conv_2 = tf.nn.conv2d(max_pool_1, conv2_weights, [1, 1, 1, 1], padding='SAME')
    relu_2 = tf.nn.relu(conv_2 + conv2_biases)
    max_pool_2 = tf.nn.max_pool(relu_2, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1],padding = 'SAME')
    
    conv_3 = tf.nn.conv2d(max_pool_2, conv3_weights, [1, 1, 1, 1], padding='SAME')
    relu_3 = tf.nn.relu(conv_3 + conv3_biases)
    max_pool_3 = tf.nn.max_pool(relu_3, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1],padding = 'SAME')
    
    conv_4 = tf.nn.conv2d(max_pool_3, conv4_weights, [1, 1, 1, 1], padding='SAME')
    relu_4 = tf.nn.relu(conv_4 + conv4_biases)
    max_pool_4 = tf.nn.max_pool(relu_4, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1],padding = 'SAME')
    
    shape = max_pool_4.get_shape().as_list()
    reshape = tf.reshape(max_pool_4, [shape[0], shape[1] * shape[2] * shape[3]])
    full_layer = tf.nn.relu(tf.matmul(reshape, full_weights) + full_biases)
    softmax_layer = tf.nn.dropout(full_layer, keep_prob)
    return tf.matmul(softmax_layer, softmax_weights) + softmax_biases
  
  classify_prediction = tf.nn.softmax(model(tf_classify_dataset))
  saver = tf.train.Saver()

In [8]:
num_loop = math.ceil(testDataSet.shape[0]/1.0/calculate_image_per_times)
with tf.Session(graph = graph) as session:
    tf.initialize_all_variables().run()
    saver.restore(session,'conv_layer4_param9-4000')
    for step in xrange(int(num_loop)):
        start = step * calculate_image_per_times
        end = (step + 1) * calculate_image_per_times 
        subset = testDataSet[start:end , :, :, :]
        feed_dict = {tf_classify_dataset : subset ,keep_prob : 1}
        label_result[start:end] = session.run(classify_prediction, feed_dict = feed_dict)


In [16]:
print label_result[6000]
count = 0
for step in xrange(int(num_loop)):
    if label_result[step, 0] > label_result[step, 1] :
        #print step 
        count += 1
accuracy = 1- count /num_loop
print "accuracy = ",accuracy

[ 0.18602404  0.81397593]
accuracy =  0.864197530864


In [17]:
import shutil
#testFolders = ['/Users/meicg/Documents/MATLAB/label_64_1'];
#originalFolder = ['/Users/meicg/Documents/MATLAB/label_1']
#classifyFolders = ['/Users/meicg/Documents/MATLAB/auto_classify_0',
#                   '/Users/meicg/Documents/MATLAB/auto_classify_1']
def classifyTestImages(small_data_folders, original_data_folder, classify_folders ):
    print small_data_folders 
    flag = 0
    for image in os.listdir(small_data_folders[0]):
        image_file_origin  = os.path.join(original_data_folder[0], image)
        image_file_destination_0 = os.path.join(classify_folders[0], image )
        image_file_destination_1 = os.path.join(classify_folders[1], image )
        if label_result[flag][0] >=0.5 :
            shutil.copyfile(image_file_origin ,image_file_destination_0)
        elif label_result[flag][1] >0.5 :
            shutil.copyfile(image_file_origin ,image_file_destination_1)
        flag += 1
    return flag

flag2 = classifyTestImages(testFolders, originalFolder,classifyFolders)

['/Users/meicg/Documents/MATLAB/label_64_1']


In [9]:
print flag2

NameError: name 'flag2' is not defined

In [20]:
from PIL import Image
im = Image.open('/Users/meicg/Documents/MATLAB/label_0/27411_3.JPG')
out = im.resize((64,64))

In [22]:
out.save('test.jpg')